In [3]:
%pip install awscli boto3


  Using cached docutils-0.19-py3-none-any.whl.metadata (2.7 kB)
  Using cached rsa-4.7.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   ------ --------------------------------- 0.8/4.7 MB 4.8 MB/s eta 0:00:01
   --------------- ------------------------ 1.8/4.7 MB 5.0 MB/s eta 0:00:01
   ---------------------- ----------------- 2.6/4.7 MB 4.4 MB/s eta 0:00:01
   ----------------------------- ---------- 3.4/4.7 MB 4.4 MB/s eta 0:00:01
   ----------------------------------- ---- 4.2/4.7 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 4.7/4.7 MB 3.9 MB/s  0:00:01
   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/14.0 MB 4.2 MB/s eta 0:00:04
   ----- ------

In [10]:
%pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl (12.9 MB)

   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0

In [4]:
%aws --version

UsageError: Line magic function `%aws` not found.


In [6]:
import boto3

In [7]:
client=boto3.client("rekognition",region_name="ap-south-1")

In [ ]:
rekognition = boto3.client('rekognition')

with open('./mi.jpeg', 'rb') as image_file:
    image_bytes = image_file.read()


response = rekognition.recognize_celebrities(Image={'Bytes': image_bytes})

face_response = rekognition.detect_faces(Image={'Bytes': image_bytes}, Attributes=['ALL'])

if response['CelebrityFaces']:
    for celeb in response['CelebrityFaces']:
        print("CB Name:", celeb['Name'])
else:
    print("No celebrity detected.")

if face_response['FaceDetails']:
    for face in face_response['FaceDetails']:
        print("Gender:", face['Gender']['Value'])

        sorted_emotions = sorted(face['Emotions'], key=lambda x: x['Confidence'], reverse=True)
        print("Dominant Emotion:", sorted_emotions[0]['Type'])

else:
    print("No face detected.")


face_response

In [ ]:
import cv2
from cv2.data import haarcascades
from cv2 import CascadeClassifier

modelPath=haarcascades+"haarcascade_frontalface_default.xml"
app=CascadeClassifier(modelPath)

camera=cv2.VideoCapture(0)

while True :
    status,image=camera.read()
    faces=app.detectMultiScale(image,1.3,5)
    for face in faces:
        x,y,w,h=face
        image=cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,255),2)

    cv2.imshow("My Image",image)
    if cv2.waitKey(0)==ord("q"):
        break

: 

In [1]:
import boto3
import cv2
from cv2.data import haarcascades

# AWS Rekognition client
rekognition = boto3.client('rekognition')

# Load image
image_path = './csk.jpeg'
with open(image_path, 'rb') as image_file:
    image_bytes = image_file.read()

# Detect celebrities
response = rekognition.recognize_celebrities(Image={'Bytes': image_bytes})

# Load OpenCV image
image = cv2.imread(image_path)
h, w, _ = image.shape

# Draw rectangles and celebrity names
for celeb in response['CelebrityFaces']:
    box = celeb['Face']['BoundingBox']  # relative coordinates
    left = int(box['Left'] * w)
    top = int(box['Top'] * h)
    width = int(box['Width'] * w)
    height = int(box['Height'] * h)

    # Draw rectangle
    cv2.rectangle(image, (left, top), (left + width, top + height), (0, 255, 0), 2)

    # Draw name in smaller font on left side
    name_position = (left, top - 15)  # slightly below top-left corner
    cv2.putText(image, celeb['Name'], name_position,
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

# Show final image
cv2.imshow("Celebrity Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
